<a href="https://colab.research.google.com/github/massone99/visione_artificiale_colab_notebooks/blob/main/GroceryProductDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Grocery Product Detection**
Obiettivo dell’esercitazione è la realizzazione di un sistema di **localizzazione di prodotti negli scaffali dei supermercati**. Per ciascun prodotto è disponibile un singolo template.

L'approccio adottato è basato su **template matching rigido con feature**; viene fatta scorrere sull'immagine in input una finestra di dimensioni coincidenti con quelle del template e per ciascuna sottofinestra viene valutata la compatibilità con il template (colore e descrittori locali).

La ricerca deve prevedere un’analisi **multiscala** perché la dimensione del template non coincide esattamente con la dimensione del prodotto nelle immagini degli scaffali.

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsProductDetection/Detection.png width="800">


# **Import delle librerie**
È necessario ora eseguire l'import delle librerie utilizzate durante l'esercitazione.
Per questa esercitazione è necessario inoltre utilizzare una versione specifica della libreria opencv perché si utilizzano feature (SIFT) che nelle versioni successive della libreria sono disponibili solo a pagamento.

In [ ]:
!pip install  opencv-python
!pip install  opencv-contrib-python
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import math
from tqdm import tqdm

# **Dataset**

Il dataset per l'esercitazione è un piccolo sottoinsieme di immagini tratte dal dataset Grozy; vengono forniti qui i template di tre prodotti e, per ciascun prodotto, qualche foto di scaffali del supermercato che contengono quel prodotto.

In [ ]:
!wget http://bias.csr.unibo.it/VR/Esercitazioni/DBs/ProductDetection.zip
!unzip /content/ProductDetection.zip
!rm /content/ProductDetection.zip


Carichiamo il template di un prodotto...

In [ ]:
product_template = cv2.imread('/content/ProductDetection/P1.png')
cv2_imshow(product_template)

...e un'immagine all'interno della quale effettuare la ricerca.

In [ ]:
input_image = cv2.imread('/content/ProductDetection/P1_1122.png')
cv2_imshow(input_image)

# **Selezione dei keypoint**

Per la rappresentazione si utilizzeranno descrittori locali SIFT calcolati in corrispondenza di keypoint individuati attraverso un processo di **dense sampling a nido d’ape**.

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsProductDetection/Sampling.png width="800">

La **spaziatura** tra i keypoint viene fissata a 16 pixel.

A ciascun keypoint viene inoltre assegnato un valore di **scala di default** pari a 3.0 che verrà poi utilizzato per il calcolo del descrittore associato al keypoint.

In [ ]:
import cv2

def kp_dense_sampling(img, spacing, scale):
    # Initialize an empty list to store keypoints
    points = []

    # Get the shape of the input image
    s = img.shape

    # Calculate the number of keypoints per column and row based on spacing
    points_per_col = int(s[0] / spacing)
    points_per_row = int(s[1] / spacing)

    # Calculate half of the spacing
    half_spacing = spacing / 2

    # Calculate the borders on the X and Y axes
    borderX = (s[1] - (points_per_row - 1) * spacing) / 2
    borderY = (s[0] - (points_per_col - 1) * spacing) / 2

    # Initialize the Y position for the keypoints
    posY = borderY

    # Loop through the rows (Y)
    for y in range(points_per_col):
        # Check if the current row is even or odd to adjust the X position
        if y % 2 == 0:
            posX = borderX
        else:
            posX = half_spacing + borderX

        # Loop through the columns (X) within the current row
        for x in range(points_per_row):
            # Check if the current position is within the image boundaries
            if posX <= s[1] - borderX and posY <= s[0] - borderY and posX >= 0 and posY >= 0:
                # Create a KeyPoint object and add it to the list of keypoints
                p = cv2.KeyPoint(posX, posY, scale)
                points.append(p)

            # Increment the X position by the spacing
            posX += spacing

        # Increment the Y position by the spacing
        posY += spacing

    # Return the list of keypoints
    return points

# Call the kp_dense_sampling function with specific parameters
keypoints = kp_dense_sampling(product_template, 16, 3.0)

# Create a copy of the original image for visualization
img_sampling = product_template.copy()

# Draw the keypoints on the copy of the image
cv2.drawKeypoints(product_template, keypoints, img_sampling, (0, 0, 255))

# Display the image with keypoints using OpenCV's imshow function
cv2_imshow(img_sampling)

# **Pre-selezione dei candidati in base al colore**
Al fine di velocizzare la procedura di ricerca del prodotto viene fatta una pre-selezione dei candidati sulla base dell'analisi dell'istogramma colore.

In particolare, solo le sottofinestre che presentano un istogramma colore "compatibile" con quello del template vengono ulteriormente valutate sulla base della similarità dei descrittori locali.

L'istogramma colore viene calcolato sui canali Cb e Cr dello spazio YCbCr. La compatibilità viene valutata dalla funzione `check_color_similarity` che si occupa di verificare che l'intersezione tra l'istogramma del template e quello della sottofinestra in oggetto sia superiore alla soglia prefissata `colorhist_intersection_thr`.

In [ ]:
# Define the number of bins for the color histogram
bin_count = 20

# Define a threshold for color histogram intersection for similarity check
colorhist_intersection_thr = 1.1

def compute_hist(img):
    image_ycbcr = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    # Estrai i canali Cb e Cr
    # channel_y = image_ycbcr[:, :, 0]  # Canale Y (canale 0)
    channel_cb = image_ycbcr[:, :, 1]  # Canale Cb (canale 1)
    channel_cr = image_ycbcr[:, :, 2]  # Canale Cr (canale 2)

    # Calcola l'istogramma per il canale Cb
    hist_cb = cv2.calcHist([channel_cb], [0], None, [bin_count], [0, 256])

    # Calcola l'istogramma per il canale Cr
    hist_cr = cv2.calcHist([channel_cr], [0], None, [bin_count], [0, 256])

    # Unisci i due istogrammi in un unico istogramma
    hist = np.concatenate((hist_cb, hist_cr), axis=0)

    return hist

# Function to check the color similarity between a product histogram and an image
def check_color_similarity(product_hist, img):
    # Compute the color histogram for the input image
    img_hist = compute_hist(img)

    # Calculate the intersection of histograms using element-wise minimum and sum
    intersection = np.minimum(product_hist, img_hist).sum()

    # Check if the intersection value is greater than the specified threshold
    if intersection > colorhist_intersection_thr:
        # If the intersection is above the threshold, consider the colors similar
        return True
    else:
        return False

# **Ricerca dei candidati con analisi multi-scala**
La ricerca del prodotto nell'immagine va fatta con un'analisi multiscala.

Per motivi di efficienza si preferisce non riscalare il template ma **riscalare l'immagine in input**. Questo permette di calcolare l'istogramma colore, i keypoint e i descrittori locali del template una sola volta.

Il template dunque è fisso, si riscala l'immagine in input e su ogni immagine riscalata viene fatta scorrere una **finestra di dimensione pari a quella del template**. Ad ogni step la finestra si sposta di un numero di pixel (in orizzontale e in verticale) pari a una percentuale della dimensione del template (parametro `window_step_perc`); in particolare, la percentuale va calcolata rispetto alla larghezza e all'altezza del template rispettivamente per lo spostamento in orizzontale e in verticale.

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsProductDetection/Scala.png width="800">

Si richiede di implementare la funzione `find_product_candidates` che riceve in input:

* `product_template` - l'immagine del template (BGR)
* `img` - l'immagine di input nella quale effettuare la ricerca
* `keypoints` - l'array di keypoint da utilizzare per il calcolo dei descrittori
* `scales` - l'array di fattori di scala da applicare

Gli step da implementare sono i seguenti.

**Rappresentazione del template**

A partire dall'immagine del template si deve calcolare il suo istogramma colore (funzione `compute_hist` implementata precedentemente) e l'insieme dei suoi descrittori locali.

Per il calcolo dei descrittori è necessario istanziare un estrattore SIFT come segue:

`sift = cv2.xfeatures2d.SIFT_create()`

I descrittori associati a un insieme di keypoint prefissato (`keypoints`) possono essere calcolati richiamando il metodo `sift.compute()` passandogli come parametri l'**immagine grayscale** e il set di keypoint.

**Scansione dell'immagine e confronto col template**

Per ogni sottofinestra individuata durante la scansione si dovrà:
* verificare la **similarità dell'istogramma** colore con l'istogramma del template;
* se la similarità a livello di colore è sufficiente, calcolare i **descrittori locali** e confrontarli con quelli del template, calcolando la **distanza Euclidea tra punti corrispondenti** e mediando infine la somma ottenuta sul numero di keypoint;
* se la **distanza media** tra descrittori è inferiore alla soglia prefissata (`dist_thr`), istanziare un candidato, memorizzando le coordinate della sottofinestra (top, left, bottom, right) e il valore di distanza media calcolato:

  `r = {"top": ?, "bottom": ?, "left": ?, "right": ?, "dist": ?}`
  
  **Attenzione!** Le coordinate della finestra vanno espresse rispetto all'immagine alla scala originale, tenete quindi in considerazione il fattore di scala...

In [ ]:
def compute_avg_dist(d1, d2):
  dist = 0.0
  for i in range(d1[1].shape[0]):
    dist += np.linalg.norm(d1[1][i] - d2[1][i])
  return dist/(d1[1].shape[0]*d1[1].shape[1])

def find_product_candidates(product_template, img, keypoints, scales):
    # Define the step size for sliding window and the distance threshold
    window_step_perc = 0.05
    dist_thr = 4.5
    candidates = []  # Store the candidates that meet the criteria

    # Compute the histogram of the product template (assuming you have this function)
    hist_temp = compute_hist(product_template)
    sift = cv2.SIFT_create()

    # Convert the product template to grayscale and compute SIFT descriptors
    temp_gray = cv2.cvtColor(product_template, code=cv2.COLOR_BGR2GRAY)
    temp_descriptors = sift.compute(temp_gray, keypoints)

    # Check for color similarity between the product template and the image
    if check_color_similarity(hist_temp, img):
        # Iterate over different scales of the input image
        for scale in scales:
            # Resize the input image based on the current scale
            scaled_img = cv2.resize(img, None, fx=scale, fy=scale)
            img_gray = cv2.cvtColor(scaled_img, code=cv2.COLOR_BGR2GRAY)

            # Iterate over the image with a sliding window
            for top in range(0, img_gray.shape[0], int(window_step_perc * img_gray.shape[0])):
                for left in range(0, img_gray.shape[1], int(window_step_perc * img_gray.shape[1])):
                    bottom = top + product_template.shape[0]
                    right = left + product_template.shape[1]

                    # Ensure the sliding window stays within the image boundaries
                    if bottom <= img_gray.shape[0] and right <= img_gray.shape[1]:
                        # Extract the region defined by the sliding window
                        window = img_gray[top:bottom, left:right]

                        # Compute SIFT descriptors for the current window
                        img_descriptors = sift.compute(window, keypoints)

                        # Calculate the average distance between SIFT descriptors using compute_avg_dist
                        avg_distance = compute_avg_dist(temp_descriptors, img_descriptors)

                        # Check if the distance is below the threshold
                        if avg_distance < dist_thr:
                            # Store the candidate's coordinates, scale, and distance
                            candidate = {
                                "top": top,
                                "bottom": bottom,
                                "left": left,
                                "right": right,
                                "scale": scale,
                                "dist": avg_distance
                            }
                            candidates.append(candidate)

    return candidates

scales = np.arange(0.5, 1.5, 0.25)
# Ricerca dei candidati
candidates = find_product_candidates(product_template, input_image, keypoints, scales)
# Visualizzazione dei candidati individuati
initial_candidates = input_image.copy()
for c in candidates:
  cv2.rectangle(initial_candidates, (round(c["left"]), round(c["top"])), (round(c["right"]), round(c["bottom"])), (0,0,255))
cv2_imshow(initial_candidates)

# **Semplificazione dell'output**

La procedura di ricerca precedente può rilevare più candidati corrispondenti allo stesso prodotto; per ridurre il numero di falsi positivi prodotti dall'algoritmo è necessario semplificare l'output mantenendo solo i candidati più significativi.

Si consiglia di implementare a tal fine una procedura di **soppressione dei non minimi** (avendo qui una misura di distanza). In particolare ogni candidato può essere confrontato con tutti gli altri e, se tra quelli che si **sovrappongono ad esso per più del 50%** (verificare boundind box) se ne trova uno con distanza inferiore, il candidato in esame va rimosso dal risultato.

In [ ]:
def non_minima_suppression(candidates):
  result = []
  # TODO
  return result

# Soppressione dei non minimi
result = non_minima_suppression(candidates)
# Visualizzazione del risultato finale
final_candidates = initial_candidates.copy()
for c in result:
  cv2.rectangle(final_candidates, (round(c["left"]), round(c["top"])), (round(c["right"]), round(c["bottom"])), (0,255,0), thickness = 3)
cv2_imshow(final_candidates)